# 데이터 파일 전처리

## #01. 패키지 참조

In [2]:
import sys
import pandas as pd
import numpy as np
from pandas import read_excel, DataFrame, pivot_table, melt, crosstab, get_dummies, merge
from matplotlib import pyplot as plt
from sklearn.impute import SimpleImputer
from labelencoder import labelencoder
import re

In [3]:
origin = read_excel("#1.musinsadata_original.xlsx")
origin

,상품번호,상품명,분류,종류,성별,가격,조회수(18세이하),조회수(19~23세),조회수(24~28세),조회수(29~33세),...,구매수(18세이하),구매수(19~23세),구매수(24~28세),구매수(29~33세),구매수(34~39세),구매수(40세이상),남성 구매수,여성 구매수,비로그인_p,총 구매수
0,2037173,베이직 푸퍼_SPJPE11C11,숏패딩/숏헤비 아우터,아우터,남성,62.91,51997,75898,53095,33623,...,9696,20758,15730,8884,6016,14698,50561,25200,21,75782
1,1163169,삼바 OG - 화이트:블랙 / B75806,패션스니커즈화,스니커즈,공용,139.00,13222,29429,31964,20633,...,234,2032,4139,3927,2582,3114,5330,10689,9,16028
2,1178135,삼바 OG - 블랙:화이트 / B75807,패션스니커즈화,스니커즈,공용,139.00,7774,31209,35733,21648,...,105,1916,4382,3709,2220,2703,4020,11012,3,15035
3,452483,솔리드 무톤 자켓,무스탕/퍼,아우터,공용,169.00,19451,45648,26254,13786,...,399,3352,2528,1431,589,916,7301,1913,1,9215
4,2782764,OVERSIZED CASHMERE BALMACAAN COAT BLACK,겨울 싱글 코트,아우터,남성,263.20,2529,21990,31855,21796,...,21,888,1849,1381,391,265,4213,581,1,4795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7678,2697817,[세트]블룸 트레이닝 셋업-그레이,맨투맨/스웨트셔츠,상의,공용,69.90,41,171,242,275,...,1,31,37,56,60,89,103,171,0,274
7679,2757362,TISNVRHT Hoodie Black,후드 티셔츠,상의,공용,38.00,1365,1488,1539,1102,...,34,55,66,59,20,35,203,66,0,269
7680,3220337,[리뉴얼] 나일론 카고 밴딩팬츠_차콜_PL170,트레이닝/조거 팬츠,바지,공용,55.00,553,785,579,452,...,129,584,787,486,154,266,1433,972,1,2406
7681,3436582,블랙 와이드핏 밴딩 윈터 테리 니트 팬츠 (TNPA3F104BK),코튼 팬츠,바지,남성,66.75,21,52,191,418,...,1,1,5,33,38,26,99,5,0,104


1. 수치형 데이터 : 수치조정 (조회수 0.001, 누적판매 0.001 < 이상치 확인후 정리, 상품가격 0.001, 연령수치 0.01, 남자선호비율), 누락값, 이상치 설정
2. 범수형 데이터 : 더미 변수(선호성별), 라벨 인코딩(종류, 분류, 선호연령, 성별, 제품소재(% 수치 제거후)  ) >> category
3. 추가 열: 순위(누락값 및 중복값 제거후 생성), 매출 (각 상품별 총 매출)
4. 제거 열: 여자 선호비율, 색상

## #02. 라벨링 변환 (종류, 분류, 성별)

### [1] 데이터 가져오고 결측치 제거

In [6]:
label_df = origin.filter(['상품명','분류','종류','성별'])
label_df

,상품명,분류,종류,성별
0,베이직 푸퍼_SPJPE11C11,숏패딩/숏헤비 아우터,아우터,남성
1,삼바 OG - 화이트:블랙 / B75806,패션스니커즈화,스니커즈,공용
2,삼바 OG - 블랙:화이트 / B75807,패션스니커즈화,스니커즈,공용
3,솔리드 무톤 자켓,무스탕/퍼,아우터,공용
4,OVERSIZED CASHMERE BALMACAAN COAT BLACK,겨울 싱글 코트,아우터,남성
...,...,...,...,...
7678,[세트]블룸 트레이닝 셋업-그레이,맨투맨/스웨트셔츠,상의,공용
7679,TISNVRHT Hoodie Black,후드 티셔츠,상의,공용
7680,[리뉴얼] 나일론 카고 밴딩팬츠_차콜_PL170,트레이닝/조거 팬츠,바지,공용
7681,블랙 와이드핏 밴딩 윈터 테리 니트 팬츠 (TNPA3F104BK),코튼 팬츠,바지,남성


In [9]:
label_df = label_df.dropna()
label_df

,상품명,분류,종류,성별
0,베이직 푸퍼_SPJPE11C11,숏패딩/숏헤비 아우터,아우터,남성
1,삼바 OG - 화이트:블랙 / B75806,패션스니커즈화,스니커즈,공용
2,삼바 OG - 블랙:화이트 / B75807,패션스니커즈화,스니커즈,공용
3,솔리드 무톤 자켓,무스탕/퍼,아우터,공용
4,OVERSIZED CASHMERE BALMACAAN COAT BLACK,겨울 싱글 코트,아우터,남성
...,...,...,...,...
7678,[세트]블룸 트레이닝 셋업-그레이,맨투맨/스웨트셔츠,상의,공용
7679,TISNVRHT Hoodie Black,후드 티셔츠,상의,공용
7680,[리뉴얼] 나일론 카고 밴딩팬츠_차콜_PL170,트레이닝/조거 팬츠,바지,공용
7681,블랙 와이드핏 밴딩 윈터 테리 니트 팬츠 (TNPA3F104BK),코튼 팬츠,바지,남성


### [2] 데이터 라벨링하기

In [10]:
label_encoder = labelencoder(label_df)
select = ['분류','종류','성별']
label_df = label_encoder.labelling(select)
label_df

[분류]
['LT/라이더스 재킷', '가구', '겨울 기타 코트', '겨울 더블 코트', '겨울 싱글 코트', '구두', '기구/용품/장비', '기타 모자', '기타 바지', '기타 상의', '기타 스니커즈', '기타 신발', '기타 아우터', '기타 액세서리', '긴CW매 티셔츠', '나일론/코치 재킷', '남성 속옷', '넥타이', '니트/스웨터', '다이어트/헬스', '데님 팬츠', '덴탈케어', '디지털', '로퍼', '롱스커트', '롱패딩/롱헤비 아우터', '맨투맨/스웨트셔츠', '머플러', '메신저/크로스 백', '모카신/보트 슈즈', '목걸이/펜던트', '무스탕/퍼', '미니 원피스', '미니스커트', '미디 원피스', '미디스커트', '미용 CW품', '민CW매 티셔츠', '바디케어', '반CW매 티셔츠', '반려동물 의류', '반지', '백팩', '버킷/사파리햇', '베스트', '베이스 메이크업', '벨트', '부츠', '뷰티 디바이스', '브리프케이스', '블로퍼', '블루종/MA-1', '비니', '사파리/헌팅 재킷', '상의', '상하의세트', '샌들', '생활용품', '선글라스', '셔츠/블라우스', '숄더백', '숏 팬츠', '숏패딩/숏헤비 아우터', '수영복/비치웨어', '슈트 팬츠/슬랙스', '슈트/블레이저 재킷', '스카프/반다나', '스킨케어', '스타디움 재킷', '스포츠가방', '스포츠모자', '스포츠신발', '스포츠잡화', '슬리퍼', '아노락 재킷', '아우터', '아트', '안경', '양말', '에코백', '여성 속옷 상의', '여성 속옷 세트', '여성 속옷 하의', '웨이스트 백', '장갑', '점프 슈트/오버올', '조명', '지갑/머니클립', '카디건', '캐리어', '캔버스/단화', '캠핑용품', '캡/야구 모자', '코튼 팬츠', '쿼츠 아날로그', '크로스백', '클러치 백', '클렌징', '키링/키케이스', '토트백', '트러커 재킷', '트레이닝 재킷', '트레이닝/조거 팬츠', '트루퍼', '티켓', 

,상품명,분류,종류,성별
0,베이직 푸퍼_SPJPE11C11,62,14,1
1,삼바 OG - 화이트:블랙 / B75806,108,9,0
2,삼바 OG - 블랙:화이트 / B75807,108,9,0
3,솔리드 무톤 자켓,31,14,0
4,OVERSIZED CASHMERE BALMACAAN COAT BLACK,4,14,1
...,...,...,...,...
7678,[세트]블룸 트레이닝 셋업-그레이,26,6,0
7679,TISNVRHT Hoodie Black,122,6,0
7680,[리뉴얼] 나일론 카고 밴딩팬츠_차콜_PL170,102,3,0
7681,블랙 와이드핏 밴딩 윈터 테리 니트 팬츠 (TNPA3F104BK),93,3,1


### [3] 원본데이터와 결합하기

#### (1) 원본에서 중복되는 데이터 삭제

In [11]:
origin = origin.drop(['분류','종류','성별'],axis=1)
origin

,상품번호,상품명,가격,조회수(18세이하),조회수(19~23세),조회수(24~28세),조회수(29~33세),조회수(34~39세),조회수(40세이상),비로그인_v,...,구매수(18세이하),구매수(19~23세),구매수(24~28세),구매수(29~33세),구매수(34~39세),구매수(40세이상),남성 구매수,여성 구매수,비로그인_p,총 구매수
0,2037173,베이직 푸퍼_SPJPE11C11,62.91,51997,75898,53095,33623,21268,46320,255816,...,9696,20758,15730,8884,6016,14698,50561,25200,21,75782
1,1163169,삼바 OG - 화이트:블랙 / B75806,139.00,13222,29429,31964,20633,13469,26030,22618,...,234,2032,4139,3927,2582,3114,5330,10689,9,16028
2,1178135,삼바 OG - 블랙:화이트 / B75807,139.00,7774,31209,35733,21648,12870,22983,24870,...,105,1916,4382,3709,2220,2703,4020,11012,3,15035
3,452483,솔리드 무톤 자켓,169.00,19451,45648,26254,13786,7169,12378,12905,...,399,3352,2528,1431,589,916,7301,1913,1,9215
4,2782764,OVERSIZED CASHMERE BALMACAAN COAT BLACK,263.20,2529,21990,31855,21796,9003,6575,10685,...,21,888,1849,1381,391,265,4213,581,1,4795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7678,2697817,[세트]블룸 트레이닝 셋업-그레이,69.90,41,171,242,275,362,735,475,...,1,31,37,56,60,89,103,171,0,274
7679,2757362,TISNVRHT Hoodie Black,38.00,1365,1488,1539,1102,560,674,452,...,34,55,66,59,20,35,203,66,0,269
7680,3220337,[리뉴얼] 나일론 카고 밴딩팬츠_차콜_PL170,55.00,553,785,579,452,292,490,666,...,129,584,787,486,154,266,1433,972,1,2406
7681,3436582,블랙 와이드핏 밴딩 윈터 테리 니트 팬츠 (TNPA3F104BK),66.75,21,52,191,418,383,244,791,...,1,1,5,33,38,26,99,5,0,104


#### (2) 데이터 결합하기

In [15]:
merged_df = pd.concat([origin, label_df], ignore_index=True)
merged_df = merged_df.drop_duplicates(subset='상품번호')
merged_df

,상품번호,상품명,가격,조회수(18세이하),조회수(19~23세),조회수(24~28세),조회수(29~33세),조회수(34~39세),조회수(40세이상),비로그인_v,...,구매수(29~33세),구매수(34~39세),구매수(40세이상),남성 구매수,여성 구매수,비로그인_p,총 구매수,분류,종류,성별
0,2037173.0,베이직 푸퍼_SPJPE11C11,62.91,51997.0,75898.0,53095.0,33623.0,21268.0,46320.0,255816.0,...,8884.0,6016.0,14698.0,50561.0,25200.0,21.0,75782.0,NaN,NaN,NaN
1,1163169.0,삼바 OG - 화이트:블랙 / B75806,139.00,13222.0,29429.0,31964.0,20633.0,13469.0,26030.0,22618.0,...,3927.0,2582.0,3114.0,5330.0,10689.0,9.0,16028.0,NaN,NaN,NaN
2,1178135.0,삼바 OG - 블랙:화이트 / B75807,139.00,7774.0,31209.0,35733.0,21648.0,12870.0,22983.0,24870.0,...,3709.0,2220.0,2703.0,4020.0,11012.0,3.0,15035.0,NaN,NaN,NaN
3,452483.0,솔리드 무톤 자켓,169.00,19451.0,45648.0,26254.0,13786.0,7169.0,12378.0,12905.0,...,1431.0,589.0,916.0,7301.0,1913.0,1.0,9215.0,NaN,NaN,NaN
4,2782764.0,OVERSIZED CASHMERE BALMACAAN COAT BLACK,263.20,2529.0,21990.0,31855.0,21796.0,9003.0,6575.0,10685.0,...,1381.0,391.0,265.0,4213.0,581.0,1.0,4795.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7679,2757362.0,TISNVRHT Hoodie Black,38.00,1365.0,1488.0,1539.0,1102.0,560.0,674.0,452.0,...,59.0,20.0,35.0,203.0,66.0,0.0,269.0,NaN,NaN,NaN
7680,3220337.0,[리뉴얼] 나일론 카고 밴딩팬츠_차콜_PL170,55.00,553.0,785.0,579.0,452.0,292.0,490.0,666.0,...,486.0,154.0,266.0,1433.0,972.0,1.0,2406.0,NaN,NaN,NaN
7681,3436582.0,블랙 와이드핏 밴딩 윈터 테리 니트 팬츠 (TNPA3F104BK),66.75,21.0,52.0,191.0,418.0,383.0,244.0,791.0,...,33.0,38.0,26.0,99.0,5.0,0.0,104.0,NaN,NaN,NaN
7682,2772719.0,유틸리티 후드 숏 패딩-베이지,89.60,201.0,1362.0,5834.0,7476.0,5740.0,4738.0,1802.0,...,74.0,57.0,62.0,163.0,92.0,1.0,256.0,NaN,NaN,NaN
